In [1]:
import torch, time
from transformers import AutoTokenizer, AutoModelForCausalLM
# Import both functions from your parser
from tree_parser import repo_scan_parser, build_mamba_prompt
from pure_mamba_audit import PureMambaAuditor

/home/quinyang/whole_codebase_auditor/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration
MODEL_ID = "state-spaces/mamba-2.8b-hf" 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
auditor = PureMambaAuditor()

⚙️  Initializing Pure Mamba WCA on cuda...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [05:41<00:00, 113.98s/it]


In [4]:
# 1. Scan Files (Using your existing code)
print("🚀 Starting Repo Scan...")
scanned_files = repo_scan_parser()

🚀 Starting Repo Scan...
🔹 Authenticate with GitHub...
🔹 Fetching file list from branch: V2.1...
Found 15 files in the repository.

⏭️ Skipping unsupported file: .env
⏭️ Skipping unsupported file: .gitignore
✅ Loaded grammar: CPP
🔹 Processing file (3/15): HPC/heston_gpu.cu
✅ Loaded grammar: PYTHON
🔹 Processing file (4/15): HPC/heston_server.py
🔹 Processing file (5/15): HPC/stock_calibration.py
🔹 Processing file (6/15): ML/generate_training_data.py
🔹 Processing file (7/15): ML/improved_parameter_variation.py
🔹 Processing file (8/15): ML/ml_data_loader.py
🔹 Processing file (9/15): ML/ml_model.py
⏭️ Skipping unsupported file: ML/mlp.ipynb
⏭️ Skipping unsupported file: README.md
⏭️ Skipping known massive vendor file: include/json.hpp
⏭️ Skipping unsupported file: requirements.txt
⏭️ Skipping unsupported file: training_data/metadata.json
⏭️ Skipping unsupported file: training_data/normalization_stats.json

✅ Scan Complete. Processed 7 files.
📊 Total Codebase Size: 90.91 KB


In [5]:
# 2. Build the Giant Prompt (Using your existing code)
# This automatically handles skipping json.hpp, adding XML tags, and appending the Query.
print("\n🔗 Stitching Mamba Context...")
huge_prompt = build_mamba_prompt(scanned_files)

print(f"📦 Prepared Prompt Length: {len(huge_prompt):,} characters")


🔗 Stitching Mamba Context...
📦 Prepared Prompt Length: 93,834 characters


In [6]:
# 3. Run Inference
result = auditor.audit_codebase(huge_prompt)


🧠 Tokenizing entire codebase...
📊 Total Input Size: 27,741 tokens
🕵️‍♂️ WCA is auditing the entire repo in one pass... (Hold tight)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [7]:
print("\n" + "="*50)
print("🛡️  PURE MAMBA AUDIT REPORT")
print("="*50)
print(result)


🛡️  PURE MAMBA AUDIT REPORT
hi saidPrep sense IBl...

hpry page?


##



